In [1]:
import os
import sys

import logging
import re
import pathlib

import datetime

import numpy as np
import pandas as pd
from scipy.stats import pearsonr

import matplotlib.pyplot as plt

from pyplink import PyPlink
import seaborn as sns

import statsmodels.api as sm

import matplotlib.patches as patches
import matplotlib

from basic_tools import *

"""

jupyter nbconvert 5_association.ipynb --to script
python 5_association.py albumin 2 0

for i in {00..101};do python 5_association.py $i 2 1;done


for i in {00..10};do python 5_association.py $i 2 0;done
for i in {11..20};do python 5_association.py $i 2 0;done
for i in {21..30};do python 5_association.py $i 2 0;done
for i in {31..40};do python 5_association.py $i 2 0;done
for i in {41..50};do python 5_association.py $i 2 0;done
for i in {51..60};do python 5_association.py $i 2 0;done
for i in {61..70};do python 5_association.py $i 2 0;done
for i in {71..80};do python 5_association.py $i 2 0;done
for i in {91..90};do python 5_association.py $i 2 0;done
for i in {91..101};do python 5_association.py $i 2 0;done
for i in {51..60};do python 5_association.py $i 2 0;done
for i in {51..60};do python 5_association.py $i 2 0;done
for i in {51..60};do python 5_association.py $i 2 0;done

for i in {00..101};do python 5_association.py $i 1 0,2;done
for i in {00..101};do python 5_association.py $i 1 1;done

for i in {00..20};do python 5_association.py $i 1 1;done
for i in {21..40};do python 5_association.py $i 1 1;done
for i in {41..60};do python 5_association.py $i 1 1;done
for i in {61..80};do python 5_association.py $i 1 1;done
for i in {81..101};do python 5_association.py $i 1 1;done

"""

'\n\njupyter nbconvert 5_association.ipynb --to script\npython 5_association.py albumin 2 0\n\nfor i in {00..101};do python 5_association.py $i 2 1;done\n\n\nfor i in {00..10};do python 5_association.py $i 2 0;done\nfor i in {11..20};do python 5_association.py $i 2 0;done\nfor i in {21..30};do python 5_association.py $i 2 0;done\nfor i in {31..40};do python 5_association.py $i 2 0;done\nfor i in {41..50};do python 5_association.py $i 2 0;done\nfor i in {51..60};do python 5_association.py $i 2 0;done\nfor i in {61..70};do python 5_association.py $i 2 0;done\nfor i in {71..80};do python 5_association.py $i 2 0;done\nfor i in {91..90};do python 5_association.py $i 2 0;done\nfor i in {91..101};do python 5_association.py $i 2 0;done\nfor i in {51..60};do python 5_association.py $i 2 0;done\nfor i in {51..60};do python 5_association.py $i 2 0;done\nfor i in {51..60};do python 5_association.py $i 2 0;done\n\nfor i in {00..101};do python 5_association.py $i 1 0,2;done\nfor i in {00..101};do py

In [2]:
plink_KCHIP_HLA_AA_SNP_1000G=PyPlink(plink_KCHIP_HLA_AA_SNP_1000G_path)
plink_KCHIP_HLA_AA_SNP_1000G_fam=plink_KCHIP_HLA_AA_SNP_1000G.get_fam().astype({'fid':str,'iid':str}).rename(columns={'fid':'FID','iid':'IID'})
plink_KCHIP_HLA_AA_SNP_1000G_bim=plink_KCHIP_HLA_AA_SNP_1000G.get_bim()

In [3]:
phenotypes=pd.read_csv(pheno_all_file_path,sep='\t')
phenotypes=phenotypes.set_index('ID').loc[plink_KCHIP_HLA_AA_SNP_1000G_fam['IID']]

binary_continuous_traits=sorted(phenotypes.columns[~phenotypes.columns.str.contains('x_ray')])    

In [98]:
#result_all[result_all['step']!=1]

In [159]:
r2_df=pd.read_csv('data/temp_r2.tsv',sep='\t',index_col=0)
#r2_df=pd.DataFrame(index=plink_KCHIP_HLA_AA_SNP_1000G_bim.index)

In [169]:
result_all=pd.read_csv('data/temp_result.tsv',sep='\t')
len(result_all['marker_name']),len(result_all[result_all['P']<5e-8]['marker_name'].unique())

(164, 51)

In [161]:
len(result_all[result_all['P']<5e-8]['marker_name'].unique())

51

In [162]:
top_markers=pd.Series(result_all[result_all['P']<5e-8]['marker_name'].unique())
len(top_markers)

51

In [163]:
top_markers_SNP=top_markers[~((top_markers.str.slice(stop=3)=='HLA')|(top_markers.str.slice(stop=3)=='AA_')|(top_markers.str.slice(stop=3)=='SNP'))]
len(top_markers_SNP)

39

In [165]:
print(1)

1


In [167]:
len(r2_df.columns)

68

In [ ]:
for marker_name in top_markers_SNP.values:
    if marker_name in r2_df.columns:
        print(marker_name,'continue')
        continue
    else:
        print(marker_name,'test')
    marker_dosage=plink_KCHIP_HLA_AA_SNP_1000G.get_geno_marker(marker_name)
    print(marker_name)
    r2_list=[]
    for idx_bim,(SNP,row) in enumerate(plink_KCHIP_HLA_AA_SNP_1000G_bim.iterrows()):
        r2=pearsonr(marker_dosage,plink_KCHIP_HLA_AA_SNP_1000G.get_geno_marker(SNP))[0]**2
        r2_list.append(r2)
    
    r2_df[marker_name]=r2_list
    
    #r2_df_temp=pd.DataFrame(r2_list,index=plink_KCHIP_HLA_AA_SNP_1000G_bim.index)
    #r2_df['here']=marker_name
    #r2_df=r2_df.sort_values(0,ascending=False)
    
    #r2_df_dict[marker_name]=r2_df

6:34172055_C/A test
6:34172055_C/A
6:34205465_G/GGAGCCC test
6:34205465_G/GGAGCCC
6:34205822_T/C continue
6:32632237_C/T test
6:32632237_C/T
6:31351530_T/C test
6:31351530_T/C
6:34178764_A/C continue
6:32449851_T/C test
6:32449851_T/C
6:29190624_G/A continue
6:30737591_G/C test
6:30737591_G/C
6:32188909_C/T continue
6:32593432_G/A continue
6:34237621_A/G test
6:34237621_A/G
AX-11365330 test
AX-11365330
6:31371397_C/T test
6:31371397_C/T
6:31358559_C/T test
6:31358559_C/T
6:34810172_T/C continue
AX-41949565 continue
6:34207474_T/TAA test
6:34207474_T/TAA
6:28660290_G/A continue
6:34967349_G/A continue
6:30754092_G/C continue
6:32149801_C/T test
6:32149801_C/T
6:29698050_A/G continue
6:34231005_T/C test
6:34231005_T/C
6:28749731_TA/T continue
6:31538497_T/A test
6:31538497_T/A
6:32638199_C/G test
6:32638199_C/G
AX-41961031 test
AX-41961031
6:33541507_A/G continue
6:34185749_C/T continue
6:32324081_A/AT test
6:32324081_A/AT


In [168]:
r2_df.to_csv('data/temp_r2.tsv',sep='\t')

In [129]:
#pd.DataFrame(r2_dict,index=plink_KCHIP_HLA_AA_SNP_1000G_bim.index)